In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# params
VOL_TARGET = 0.60
MAX_LEV = 1.0
WIN_PORTVOL = 60
COST_BPS = 10

# gate ML
# alpha = agressivité du deleveraging
ALPHA = 2.0
P_CAP = 0.80

# inv-vol weights
vol = rets.rolling(WIN_VOL).std()
w_inv = 1.0 / vol.replace(0, np.nan)
w_inv = w_inv.fillna(0.0)
w_inv = w_inv.div(w_inv.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

# baseline low_vol
q_cs = vol.quantile(0.5, axis=1)
w_low = (vol.le(q_cs, axis=0)).astype(float)
w_low = w_low.div(w_low.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

# vol targeting scalar
w_lag = w_inv.shift(1).fillna(0.0)
port_ret_gross = (w_lag * rets).sum(axis=1)
port_vol = port_ret_gross.rolling(WIN_PORTVOL).std() * np.sqrt(FREQ)
scale = (VOL_TARGET / port_vol).clip(lower=0.0, upper=MAX_LEV).fillna(0.0)

# ML gate -> between 0 and 1
# g = 1 / (1 + alpha*p)
p_use = p_stress.reindex(rets.index).fillna(method="ffill").fillna(0.0)
p_use = p_use.clip(0.0, P_CAP)
gate_ml = 1.0 / (1.0 + ALPHA * p_use)

# final weights
W_ml = w_inv.mul(scale, axis=0).mul(gate_ml, axis=0)
W_ml = W_ml.clip(lower=0.0)
W_ml = W_ml.div(W_ml.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

# backtest util
def run_bt(weights, rets, cost_bps=10):
    w = weights.shift(1).reindex(rets.index).fillna(0.0)
    w = w.div(w.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

    gross = (w * rets).sum(axis=1)
    to = w.diff().abs().sum(axis=1).fillna(0.0)
    cost = to * (cost_bps / 10000.0)

    net = gross - cost
    return net, to

def stats(r, freq=FREQ):
    r = r.dropna()
    eq = (1 + r).cumprod()
    mu = r.mean() * freq
    sig = r.std() * np.sqrt(freq)
    sharpe = mu / sig if sig > 0 else np.nan
    dd = (eq / eq.cummax() - 1.0).min()
    ann = eq.iloc[-1] ** (freq/len(r)) - 1 if len(r) > 0 else np.nan
    return float(ann), float(sig), float(sharpe), float(dd)

# compare full period
r_low, to_low = run_bt(w_low, rets, cost_bps=COST_BPS)
r_inv, to_inv = run_bt(w_inv, rets, cost_bps=COST_BPS)
r_ml,  to_ml  = run_bt(W_ml,  rets, cost_bps=COST_BPS)

rows = []
for name, r, to in [
    ("low_vol", r_low, to_low),
    ("inv_vol", r_inv, to_inv),
    ("ml_risk_gate", r_ml, to_ml),
]:
    ann, sig, sh, dd = stats(r)
    rows.append({"name": name, "ann_return": ann, "ann_vol": sig, "sharpe": sh, "max_dd": dd, "turnover": float(to.mean())})

summary = pd.DataFrame(rows).set_index("name").sort_values("sharpe", ascending=False)
print(summary)

# equity plot
plt.figure(figsize=(12,5))
plt.plot((1+r_low.fillna(0)).cumprod(), label="low_vol")
plt.plot((1+r_inv.fillna(0)).cumprod(), label="inv_vol")
plt.plot((1+r_ml.fillna(0)).cumprod(), label="ml_risk_gate")
plt.legend()
plt.title("Equity")
plt.tight_layout()
plt.show()

# diagnostics
print("avg gate:", float(gate_ml.mean()), "min gate:", float(gate_ml.min()), "max gate:", float(gate_ml.max()))


/tmp/ipython-input-2392927508.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

  p_use = p_stress.reindex(rets.index).fillna(method="ffill").fillna(0.0)

              ann_return   ann_vol    sharpe    max_dd  turnover

name                                                            
inv_vol         0.177349  0.698655  0.589589 -0.860716  0.031327

low_vol         0.102579  0.649064  0.481898 -0.816893  0.088950

ml_risk_gate   -0.039288  0.662760  0.281045 -0.860716  0.029944


![Image test 14](image/image_test_14.png)


Sur tes résultats, inv-vol est la meilleure stratégie (Sharpe ≈ 0.59), devant low-vol. 

En revanche, la stratégie ml_risk_gate fait moins bien (rendement négatif et Sharpe plus faible), même si son turnover est bas. 

Concrètement, le gate ML réduit souvent l’exposition (gate moyen ≈ 0.68), ce qui coupe une partie des phases haussières, mais ne réduit pas le drawdown (il reste au niveau d’inv-vol). Donc à ce stade, le modèle de stress agit surtout comme un “frein” qui coûte de la perf sans apporter une protection suffisante.